In [2]:
import os
import torch
import yaml
from addict import Dict
from model_lightning_seg import MyModel

# === Configura tus rutas ===
ckpt_dir = "/Users/emilio/Library/CloudStorage/Box-Box/GitHub/Breast-AI-model/experiment_5_giancarlo"
config_path = "default_config_train_seg.yaml"
onnx_out_dir = "./onnx_models"
os.makedirs(onnx_out_dir, exist_ok=True)

# === Carga el config YAML ===
with open(config_path, "r") as f:
    conf = Dict(yaml.safe_load(f))

# === Encuentra los .ckpt ===
ckpts = sorted([
    os.path.join(ckpt_dir, f)
    for f in os.listdir(ckpt_dir)
    if f.endswith(".ckpt")
])

if not ckpts:
    raise FileNotFoundError(f"No .ckpt found in {ckpt_dir}")

# === Exporta a ONNX con Lightning ===
for i, ckpt_path in enumerate(ckpts):
    print(f"📦 Exportando modelo {i+1}/{len(ckpts)}: {ckpt_path}")
    
    model = MyModel.load_from_checkpoint(
        checkpoint_path=ckpt_path,
        model_opts=conf.model_opts,
        train_par=conf.train_par,
        strict=False,
    )
    model.eval()

    # Dummy input: (1, 1, 128, 128, 128)
    dummy_input = torch.randn(1, 1, 128, 128, 128, dtype=torch.float32)

    # Ruta de salida ONNX
    onnx_path = os.path.join(onnx_out_dir, f"model_fold{i}.onnx")

    # Exportar usando Lightning
    model.to_onnx(
        file_path=onnx_path,
        input_sample=dummy_input,
        export_params=True,
        opset_version=17,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    )

    print(f"✅ Guardado: {onnx_path}\n")

print("🎉 Conversión a ONNX completada.")


📦 Exportando modelo 1/10: /Users/emilio/Library/CloudStorage/Box-Box/GitHub/Breast-AI-model/experiment_5_giancarlo/kfold_10_experiment_1.ckpt


ModuleNotFoundError: `MyModel.to_onnx()` requires `onnx` to be installed.